In [1]:

import warnings
import cv2
import time
import os
import sys
import pytesseract
import numpy as np
from datasets import DatasetDict
from PIL import Image

# zeige keine Warnungen an
warnings.filterwarnings("ignore")

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from src.ocr_pipeline import OCRPreprocessor, OCRPostProcessor
from src.utils import rotate_image, pil_to_cv, from_cv_to_pil

/usr/local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Datensatz initialisieren
dataset = DatasetDict.load_from_disk("../data/interim_rgb")

In [3]:
def ocr_preprocessor_wrapper(example: dict) -> dict:
    """Initialisiert und wendet den OCRPreprocessor an, speichert das verarbeitete Bild."""
    preprocessor = OCRPreprocessor(example['image'])
    preprocessor.cropping(buffer_size=10)
    preprocessor.contrast_stretching()
    preprocessor.to_gray()
    preprocessor.correct_skew()
    preprocessor.sharpen(kernel_type="laplace_standard")
    preprocessor.opening(kernel=(1,1), iterations=2)
    preprocessor.power_law_transform(gamma=2)
    example['image-ocr'] = preprocessor.get_image()
        
    return example

def ocr_postprocessor_wrapper(example: dict, text_column: str) -> dict:
    """Initialisiert und wendet den OCRPostProcessor auf den extrahierten Text an."""
    if example[text_column].strip():  # Prüft, ob `ocr_output` nicht leer ist
        postprocessor = OCRPostProcessor(example[text_column])
        # Anwenden verschiedener Methoden
        postprocessor.identify_language()
        postprocessor.remove_special_characters()
        postprocessor.lowercase()
        postprocessor.remove_stopwords()
        postprocessor.remove_extra_spaces()
        # Aufbereiteten OCR-Output extrahieren
        example[text_column] = postprocessor.get_text()
    else:
        example[text_column] = "no text found in document image with ocr!"
    return example

In [6]:
import io
import concurrent.futures
from concurrent.futures import ProcessPoolExecutor
from tqdm import tqdm
import logging
from typing import Callable, Dict, List
import multiprocessing
multiprocessing.set_start_method('spawn', force=True)

class OCRPipeline:
    def __init__(self, dataset: Dict) -> None:
        # Initialisierung der OCRPipeline mit einem Datensatz.
        self.dataset = dataset
        # Ein Wörterbuch zur Speicherung der geladenen OCR-Engines.
        self.engines = {}
        
    def _load_engine(self, engine_name: str) -> None:
        """Lädt die angegebene OCR-Engine, falls noch nicht geladen."""
        # Lädt OCR-Engines dynamisch, um Ressourcen effizient zu nutzen.
        if engine_name == 'easyocr' and 'easyocr' not in self.engines:
            self.engines['easyocr'] = "easyocr"
        elif engine_name == 'doctr' and 'doctr' not in self.engines:
            self.engines['doctr'] = "doctr"
    
    def _run_tesseract(self, num_proc: int) -> None:
        """Verwendet Tesseract OCR, um Text aus Bildern in einem Datensatz zu extrahieren."""
        texts = []
        with ProcessPoolExecutor(max_workers=num_proc) as executor:
            futures = [executor.submit(pytesseract.image_to_string, image) for image in self.dataset["image_ocr"]]
            # Fortschrittsbalken zur Überwachung des Prozesses.
            for future in tqdm(concurrent.futures.as_completed(futures), total=len(futures), desc="OCR Processing"):
                # Speichert das Ergebnis jeder fertigen Aufgabe.
                texts.append(future.result())
        # Fügt die extrahierten Texte als neue Spalte zum Datensatz hinzu.
        self.dataset = self.dataset.add_column("text", texts)
     
    
    def preprocess(self, preprocessor_fn: Callable, num_proc: int = 1) -> None:
        """Wendet eine Preprocessing-Funktion auf den Datensatz an."""
        self.dataset = self.dataset.map(preprocessor_fn, num_proc=num_proc)
        logging.info("OCR-Preprocessor erfolgreich auf Datensatz angewendet.")
    
    
    def extract_text(self, ocr_engines: List[str], batched: bool = True, batch_size: int = 4, num_proc: int = 1) -> None:
        """Extrahiert Text unter Verwendung der angegebenen OCR-Engines."""
        for engine in ocr_engines:

            if engine == 'tesseract':
                self._run_tesseract(num_proc=num_proc)
                logging.info("Tesseract-OCR-Engine erfolgreich auf Datensatz angewendet")
            else:
                logging.error(f"Angegebene OCR-Engine '{engine}' nicht implementiert. Nur 'tesseract', 'easyocr', 'doctr' nutzbar.")

    def postprocess(self, postprocessor_fn: Callable, text_column: str, num_proc: int = 1) -> None:
        """Wendet eine Postprocessing-Funktion auf extrahierten Text im Datensatz an."""
        self.dataset = self.dataset.map(
            postprocessor_fn,
            fn_kwargs={'text_column': text_column},
            num_proc=num_proc
        )
        logging.info("OCR-Postprocessor erfolgreich auf Datensatz angewendet.")

    def get_data(self) -> Dict:
        """Gibt den verarbeiteten Datensatz zurück."""
        return self.dataset


In [7]:
ocr_pipeline = OCRPipeline(dataset["test"])

ocr_pipeline.preprocess(preprocessor_fn=ocr_preprocessor_wrapper, num_proc=1)

Map (num_proc=2):  29%|██▉       | 153/523 [01:15<03:02,  2.03 examples/s]


RuntimeError: One of the subprocesses has abruptly died during map operation.To debug the error, disable multiprocessing.

In [ ]:
ocr_pipeline.extract_text(batched=True, batch_size=4, num_proc=1, ocr_engines = ["tesseract"])

In [ ]:
ocr_pipeline.postprocess(postprocessor_fn=ocr_postprocessor_wrapper, text_column="text", num_proc=1)

In [ ]:
test_dataset = ocr_pipeline.get_data()

In [ ]:
# Funktion, die prüft, ob der Text leer ist
def is_empty_string(example):
    return example["text"] == ""

# Zählen der leeren Strings in jedem Split
empty_counts = {}
for split in dataset.keys():
    empty_count = sum(1 for example in dataset[split] if is_empty_string(example))
    empty_counts[split] = empty_count

# Ausgabe der Ergebnisse
for split, count in empty_counts.items():
    print(f"Anzahl der leeren Strings im '{split}'-Split: {count}")

In [ ]:
processed_dataset.save_to_disk("../data/processed")